In [1]:
# Import necessary libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# GitHub raw content link
github_link = 'https://raw.githubusercontent.com/hippoghost/test1/master/data.csv'

# Load DataFrame
df = pd.read_csv(github_link)

In [3]:
# Import necessary libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.Label('Select Category:'),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in ['All'] + list(df['Category'].unique())],
        value='All'
    ),
    html.Label('Select Subcategory:'),
    dcc.Dropdown(id='subcategory-dropdown', options=[{'label': 'All', 'value': 'All'}], multi=True),
    html.Label('Select Year:'),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in ['All'] + list(df.columns[2:])],
        value=['All'],
        multi=True
    ),
    dcc.Graph(id='result-plot'),
    dcc.Graph(id='comparison-plot')
])

# Define callback to update subcategories dropdown based on selected category
@app.callback(
    Output('subcategory-dropdown', 'options'),
    [Input('category-dropdown', 'value')]
)
def update_subcategories(selected_category):
    subcategories = ['All'] + list(df[df['Category'] == selected_category]['Subcategory'].unique())
    return [{'label': subcategory, 'value': subcategory} for subcategory in subcategories]

# Define callback to update the plot based on selected options
@app.callback(
    [Output('result-plot', 'figure'),
     Output('comparison-plot', 'figure')],
    [Input('category-dropdown', 'value'),
     Input('subcategory-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_plot(selected_category, selected_subcategories, selected_years):
    filtered_df = df.copy()

    # Initialize result_fig and comparison_fig
    result_fig = go.Figure()
    
    # Check if there are selected years
    if selected_years and 'All' not in selected_years:
        comparison_fig = make_subplots(rows=len(selected_years), cols=1, shared_xaxes=True,
                                       subplot_titles=selected_years)

        if selected_category != 'All':
            filtered_df = filtered_df[filtered_df['Category'] == selected_category]

        if selected_subcategories and 'All' not in selected_subcategories:
            filtered_df = filtered_df[filtered_df['Subcategory'].isin(selected_subcategories)]

        # Filter the DataFrame based on selected years
        selected_years = [col for col in selected_years if col in df.columns]
        filtered_df = filtered_df[['Category', 'Subcategory'] + selected_years]

        if not filtered_df.empty:
            # Calculate percentage columns for each selected year
            for year in selected_years:
                filtered_df[f'{year}_percentage'] = (filtered_df[year] / filtered_df[year].sum()) * 100
                filtered_df[f'{year}_count'] = filtered_df[year]

            # Bar plot for selected category, subcategories, and years
            result_fig = px.bar(filtered_df.melt(id_vars=['Category', 'Subcategory'], var_name='Year'),
                                x='Subcategory', y='value', color='Subcategory',
                                facet_col='Year', facet_col_wrap=1,
                                labels={'Subcategory': 'Subcategory', 'value': 'Percentage'},
                                title='Accident Percentage by Subcategory')

            # Comparative plot using scatter plot for each selected year
            for i, year in enumerate(selected_years):
                scatter_fig = px.scatter(filtered_df, x='Subcategory', y=f'{year}_percentage',
                                         color='Subcategory',
                                         labels={'Subcategory': 'Subcategory', f'{year}_percentage': 'Percentage'},
                                         title=f'Comparative Accident Percentage by Subcategory for {year}',
                                         hover_data=[f'{year}_percentage', f'{year}_count'])

                # Add each scatter plot as a trace to the subplot
                for trace in scatter_fig['data']:
                    comparison_fig.add_trace(trace, row=i + 1, col=1)

                # Print insights about the comparison plot
                print(f"Insights for {year} comparison plot:")
                print(filtered_df[[year, f'{year}_percentage', f'{year}_count']])

            comparison_fig.update_layout(height=500 * len(selected_years), showlegend=False)

        else:
            # If filtered_df is empty, append an empty bar plot to the list
            comparison_fig = go.Figure()
            comparison_fig.add_trace(go.Scatter(x=[], y=[], mode='markers', marker=dict(color='white')),
                                     row=1, col=1)
            comparison_fig.update_layout(height=500, showlegend=False)

    else:
        # If no years are selected, create an empty figure
        comparison_fig = go.Figure()

    return result_fig, comparison_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.